# PREPROCESSING

In [2]:
import pandas as pd  
import numpy as np 
import re

## 1. POSTS DATA

In [3]:
df_posts = pd.read_csv('./dataset/posts.csv')
df_posts.head()

,Post_id,User_id,Title,Post_time,Replies,Views
0,https://voz.vn/t/bitcoin-vuot-moc-60-000-usd-t...,1736979,"\n\t\t\t\t\tBitcoin vượt mốc 60.000 USD, tiến ...",2024-02-28T23:36:06+0700,623,78K
1,https://voz.vn/t/canh-bao-tuyet-%C4%91oi-khong...,2020115,\n\t\t\t\t\t[Cảnh báo] Tuyệt đối không liên hệ...,2024-07-17T11:55:20+0700,0,2K
2,https://voz.vn/t/noi-quy-box-tien-%C4%91ien-tu...,268,\n\t\t\t\t\tNội quy box Tiền điện tử - Đọc kỹ ...,2023-05-26T13:43:15+0700,0,6K
3,https://voz.vn/t/luc-nay-khong-muc-bitcoin-con...,1275125,\n\t\t\t\t\tLúc này không múc Bitcoin còn đợi ...,2023-06-15T20:27:27+0700,88K,5M
4,https://voz.vn/t/tong-hop-keo-%C4%91ao-coin-tr...,1630609,\n\t\t\t\t\tTổng hợp kèo đào coin trên Telegra...,2024-03-17T16:20:34+0700,195,19K


In [ ]:
# post.csv
# views
views = df_posts['Views'].astype(str)
for i, view in enumerate(views):
    if re.findall('K', view):
        views[i] = view[:-1] + str('000')
    elif re.findall('M', view):
        views[i] = view[:-1] + str('000000')
    elif not view.isdigit():
        digits = re.findall(r'\d+', view) 
        views[i] = ''.join(digits) if digits else None # Tìm tất cả các nhóm số
df_posts['Views'] = pd.to_numeric(views)


# replies
replies = df_posts['Replies'].astype(str)
for i, reply in enumerate(replies):
    if re.findall('K', reply):
        replies[i] = reply[:-1] + str('000')
    elif re.findall('M', reply):
        replies[i] = reply[:-1] + str('000000')
    elif not reply.isdigit():
        digits = re.findall(r'\d+', reply) 
        replies[i] = ''.join(digits) if digits else None # Tìm tất cả các nhóm số
df_posts['Replies'] = pd.to_numeric(replies)

# title
titles = df_posts['Title'].astype(str)
for i, title in enumerate(titles):
    titles[i] = title.strip()
df_posts['Title'] = titles

# id 
post_id = df_posts['Post_id'].astype(str)
for i, id in enumerate(post_id):
    post_id[i] = id.split('.')[-1][:-1]
df_posts['Post_id'] = post_id

# rename
df_posts.columns = [name.lower() for name in df_posts.columns]

In [32]:
df_posts

,post_id,user_id,title,post_time,replies,views
0,926433,1736979,"Bitcoin vượt mốc 60.000 USD, tiến gần kỷ lục",2024-02-28T23:36:06+0700,623.0,78000.0
1,986421,2020115,[Cảnh báo] Tuyệt đối không liên hệ - giao dịch...,2024-07-17T11:55:20+0700,0.0,2000.0
2,780221,268,Nội quy box Tiền điện tử - Đọc kỹ trước khi ho...,2023-05-26T13:43:15+0700,0.0,6000.0
3,792751,1275125,Lúc này không múc Bitcoin còn đợi lúc nào???,2023-06-15T20:27:27+0700,88000.0,5000000.0
4,934397,1630609,Tổng hợp kèo đào coin trên Telegram uy tín nhấ...,2024-03-17T16:20:34+0700,195.0,19000.0
...,...,...,...,...,...,...
6758,329827,1742855,[KÈO SIÊU NGON] Đào coin MNC kiếm 10$/ngày bán...,2021-06-22T12:13:26+0700,7.0,1000.0
6759,330257,835165,"MoonEdge - Kèo vừa Airdrop và IDO, còn 2 ngày ...",2021-06-22T21:54:03+0700,5.0,286.0
6760,330541,835165,BDVXP - Kèo mạng XTZ nhé! Đã check UY TÍN!,2021-06-23T11:14:24+0700,6.0,227.0
6761,328741,1744614,BOUNTY CONTEST SIÊU KHỦNG (CƠ HỘI NHẬN TỚI 10 ...,2021-06-20T20:05:09+0700,20.0,896.0


## 2. USERS DATA

In [5]:
# df_user1 and df_user_2
df_user1 = pd.read_csv('dataset/data_users_1.csv')
df_user2 = pd.read_csv('dataset/data_users_2.csv')

# df_user1
# drop 1st column
df_user1.drop(df_user1.columns[0], axis=1, inplace=True)
# profile_link
profile_links = df_user1['profile_link'].astype(str)
for i, link in enumerate(profile_links):
    profile_links[i] = 'https://voz.vn' + link
df_user1['profile_link'] = profile_links

# df_user2
# # drop 1st column
df_user2.drop(df_user2.columns[0], axis=1, inplace=True)

# num_message
num_message = df_user2['num_message'].astype(str)
for i, item in enumerate(num_message):
    digits = re.findall(r'\d+', item) 
    num_message[i] = ''.join(digits) if digits else None
df_user2['num_message'] = pd.to_numeric(num_message)

# reaction_score
reaction_score = df_user2['reaction_score'].astype(str)
for i, item in enumerate(reaction_score):
    digits = re.findall(r'\d+', item) 
    reaction_score[i] = ''.join(digits) if digits else None
df_user2['reaction_score'] = pd.to_numeric(reaction_score)

# joined_time
df_user2['joined_time'] = pd.to_datetime(df_user2['joined_time'], utc=True, unit='s')

In [6]:
# merge 
pd.set_option('display.max_rows', 500)
df_user = df_user1.set_index(['id', 'name', 'level']).join(df_user2.set_index(['id', 'name', 'level'])).reset_index()


In [10]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9329 entries, 0 to 9328
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              9329 non-null   int64              
 1   name            9328 non-null   object             
 2   level           9329 non-null   object             
 3   profile_link    9329 non-null   object             
 4   joined_time     6782 non-null   datetime64[ns, UTC]
 5   num_message     6782 non-null   float64            
 6   reaction_score  6782 non-null   float64            
 7   point           6782 non-null   float64            
dtypes: datetime64[ns, UTC](1), float64(3), int64(1), object(3)
memory usage: 583.2+ KB


In [11]:
df_user.to_csv('dataset/users.csv', index=False)

In [ ]:
# impute missing values
# only 1 row doesn't have a name -> drop
# df_user.dropna(axis=0, inplace=True, subset=['name'])

# almost admin private their account -> joined_time: oldest; num_message, reaction_score, point: higher (MICE)